<a href="https://colab.research.google.com/github/rickycamilo9/Exploring-MLB-s-2020-Season-with-Altered-Baseball-/blob/main/Juiced_Baseballs_Project_(Predictive_Model_Statcast).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pybaseball==2.2.1
%matplotlib inline
import pandas as pd; 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import seaborn as sns
import sklearn as sk
import pybaseball as pyb
from matplotlib import pyplot as plt
from pybaseball import cache
cache.enable()
import warnings; warnings.filterwarnings('ignore')

     |████████████████████████████████| 415 kB 2.9 MB/s 
     |████████████████████████████████| 291 kB 32.7 MB/s 
     |████████████████████████████████| 856 kB 34.5 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [ ]:
# exit velocity data averaged by team and season per df 

import pandas as pd

hitting_velo_2018 = pd.read_csv("hitting 2018 exit_velocity.csv")
hitting_velo_2019 = pd.read_csv("hitting 2019 exit_velocity.csv")
hitting_velo_2020 = pd.read_csv("hitting 2020 exit_velocity.csv")
hitting_velo_2021 = pd.read_csv("hitting 2021 exit_velocity.csv")


hitting_velo_2018 = hitting_velo_2018
hitting_velo_2019 = hitting_velo_2019
hitting_velo_2020 = hitting_velo_2020
hitting_velo_2021 = hitting_velo_2021



FileNotFoundError: ignored

In [ ]:
total_velo_stats = pd.concat([hitting_velo_2018, hitting_velo_2019, hitting_velo_2020 , hitting_velo_2021], axis=0)
velo_stats_2018_2019 = pd.concat([hitting_velo_2018, hitting_velo_2019,hitting_velo_2020], axis=0)

In [ ]:
total_velo_stats
velo_stats_2018_2019

total_velo_stats


In [ ]:

total_velo_stats = total_velo_stats.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
velo_stats_2018_2019 = velo_stats_2018_2019.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')


In [ ]:
total_velo_stats_copy = total_velo_stats.copy()

total_velo_stats_copy = total_velo_stats_copy.loc[:, ['season','team','max_hit_speed','avg_hit_speed', 'max_distance', 'avg_hr_distance']]

total_velo_stats_copy['max_hit_speed_2020'] = total_velo_stats_copy.sort_values(['season','team'], ascending=False).groupby('team')['max_hit_speed'].shift()
total_velo_stats_copy['avg_hit_speed_2020'] = total_velo_stats_copy.sort_values(['season','team'], ascending=False).groupby('team')['avg_hit_speed'].shift()
total_velo_stats_copy['max_distance_2020'] = total_velo_stats_copy.sort_values(['season','team'], ascending=False).groupby('team')['max_distance'].shift()
total_velo_stats_copy['avg_hr_distance_2020'] = total_velo_stats_copy.sort_values(['season','team'], ascending=False).groupby('team')['avg_hr_distance'].shift()



total_velo_stats_copy = total_velo_stats_copy.loc[total_velo_stats_copy['max_hit_speed_2020'].notnull()]
total_velo_stats_copy = total_velo_stats_copy.loc[total_velo_stats_copy['avg_hit_speed_2020'].notnull()]
total_velo_stats_copy = total_velo_stats_copy.loc[total_velo_stats_copy['max_distance_2020'].notnull()]
total_velo_stats_copy = total_velo_stats_copy.loc[total_velo_stats_copy['avg_hr_distance_2020'].notnull()]

total_velo_stats_copy

In [ ]:
total_velo_stats_copy.corr()

In [ ]:
# splitting our data for x, y training and testing

from sklearn.model_selection import train_test_split

x = total_velo_stats_copy[['max_hit_speed','avg_hit_speed', 'max_distance', 'avg_hr_distance']].values
y= total_velo_stats_copy[['max_hit_speed_2020', 'avg_hit_speed_2020', 'max_distance_2020', 'avg_hr_distance_2020']].values

print('Original Data Shape - X: {0}, Y: {1}'.format(x.shape, y.shape))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)       #test size is 20% of original


print('Train Data Shape - X{0}, Y:{1}.'.format(x_train.shape, y_train.shape) )
print('Test Data Shape - X{0}, Y:{1}.'.format(x_test.shape, y_test.shape) )

In [ ]:
#taking our training data and fitting it into our regression model 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


lr = LinearRegression()

lr.fit(x_train, y_train)

In [ ]:

y_pred = lr.predict(x_train)

print ('Mean number of hits:', x_train[:, 0].mean())
print ('Mean absolute error:', mean_absolute_error(y_pred, y_train))

In [ ]:
velo_stats_2018_2019_copy = velo_stats_2018_2019.copy()

velo_stats_2018_2019_copy = velo_stats_2018_2019_copy.loc[:,['season','team','max_hit_speed','avg_hit_speed', 'max_distance', 'avg_hr_distance']]

velo_stats_2018_2019_copy['2019_actual_max_hit_speed'] = velo_stats_2018_2019_copy.sort_values(['team', 'season'], ascending=False).groupby('team')['max_hit_speed'].shift()
velo_stats_2018_2019_copy['2019_actual_avg_hit_speed'] = velo_stats_2018_2019_copy.sort_values(['team', 'season'], ascending=False).groupby('team')['avg_hit_speed'].shift()
velo_stats_2018_2019_copy['2019_actual_max_distance'] = velo_stats_2018_2019_copy.sort_values(['team', 'season'], ascending=False).groupby('team')['max_distance'].shift()
velo_stats_2018_2019_copy['2019_actual_avg_hr_distance'] = velo_stats_2018_2019_copy.sort_values(['team', 'season'], ascending=False).groupby('team')['avg_hr_distance'].shift()

velo_stats_2018_2019_copy = velo_stats_2018_2019_copy.loc[velo_stats_2018_2019_copy['2019_actual_max_hit_speed'].notnull()]

velo_stats_2018_2019_copy


In [ ]:
x = velo_stats_2018_2019_copy[['max_hit_speed','avg_hit_speed', 'max_distance', 'avg_hr_distance' ]].values
y = velo_stats_2018_2019_copy[['2019_actual_max_hit_speed','2019_actual_avg_hit_speed', '2019_actual_max_distance', '2019_actual_avg_hr_distance' ]].values

In [ ]:
y_pred = lr.predict(x)
print('Mean of Stats:', velo_stats_2018_2019_copy.mean())
print('Mean absolute error:', mean_absolute_error(y_pred, y))

In [ ]:
velo_stats_2018_2019_copy[['predicted_max_hit_speed', 'predicted_avg_hit_speed', 'predicted_max_distance', 'predicted_avg_hr_distance', ]] = y_pred

velo_stats_2018_2019_copy['season'] = 2019

#pitch_2019_copy = pitch_2019_copy.rename(columns={'2019_actual_g':'Actual_G'}

#use sort_values to find the top predicted hits
velo_stats_2018_2019_copy = velo_stats_2018_2019_copy.loc[:,['season','team','2019_actual_max_hit_speed', '2019_actual_avg_hit_speed', '2019_actual_max_distance', '2019_actual_avg_hr_distance', 'predicted_max_hit_speed' , 'predicted_avg_hit_speed', 'predicted_max_distance', 'predicted_avg_hr_distance',  ]]

In [ ]:
predicted_2020_stats = velo_stats_2018_2019_copy.loc[:,['team', 'predicted_max_hit_speed', 'predicted_avg_hit_speed', 'predicted_max_distance', 'predicted_avg_hr_distance']]
predicted_2020_stats

In [ ]:
predicted_2020_stats.mean()